### setup

In [2]:
# needed if you're doing a new state, and needs to be done before importing dot_functions
import matplotlib
import scipy as sp

import matplotlib.animation as animation
from matplotlib.widgets import Slider, Button 
matplotlib.use("Qt5Agg")

In [3]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

In [4]:
import hyperspy.api as hs
import numpy as np
from helpers_orig import *
import helpers
import QD_state

In [5]:
im12 = hs.load("../Data/chain/12_1.3Mx_12us_1024_1.ser")
im14 = hs.load("../Data/chain/14_2.55Mx_12us_1024_1.ser")
im33 = hs.load("../Data/chain/33_910Mx_12us_1024_1.ser")
im48 = hs.load("../Data/chain/48_2.55Mx_12us_1024_1.ser")
im81 = hs.load("../Data/chain/81_1.3Mx_12us_1024_1.ser")
im92 = hs.load("../Data/chain/92_2.55Mx_12us_1024_1.ser")
im97 = hs.load("../Data/chain/97_1.8Mx_12us_1024_1.ser")
im128 = hs.load("../Data/chain/128_1.8Mx_12us_1024_1.ser")

In [6]:
all_ims = [im12, im14, im33, im48, im81, im92, im97, im128]
# show_im(im97)

In [7]:
s97 = QD_state.state(im97.data, 'chain_97', np.shape(im97))

In [8]:
s92 = QD_state.state(im92.data, 'chain_92', np.shape(im92))

In [104]:
show_im(s97.orig_image)

In [9]:
s97.segment_dots()

seg threshhold:  16907.834375
this part takes a minute
adding [ 593.5086580086579 ,  7.257575757575637 ]
closing
These are the final ponts. 

            Click to close the figure and you'll be prompted 
            if you want to try again. 
does this work? (y/n)y
heres the segmented image... not necessary but nice to see it didnt screw up.


In [10]:
show_im(s92.orig_image)

In [34]:
s92.segment_dots()

seg threshhold:  16914.603125
this part takes a minute
delete pressed
delete  (971.0, 223.5)
adding [ 1006.3225108225107 ,  242.75541125541127 ]
delete pressed
delete  (53.0, 972.0)
adding [ 50.47835497835467 ,  999.1190476190477 ]
delete pressed
delete  (310.0, 929.0)
adding [ 255.49999999999977 ,  965.8722943722944 ]
closing
These are the final ponts. 

            Click to close the figure and you'll be prompted 
            if you want to try again. 
does this work? (y/n)y
heres the segmented image... not necessary but nice to see it didnt screw up.


'current point'

As an example of where to threshhold, s97 works well with ~16890, s92 with ~16914

### segmentation 

In [14]:
from scipy.ndimage.morphology import binary_closing, binary_opening, binary_dilation, binary_erosion, distance_transform_edt
import skimage
from skimage.morphology import watershed
from skimage.segmentation import random_walker
from scipy.ndimage.measurements import center_of_mass

def segment_im2(im, threshhold):
    '''This function takes in the original image and segments it using a random walker algorithm 
    and the threshhold value given. It returns the segmented image
    (where each QD corresponds to a different color value) and the number of QDs. 
    Originally was done by a watershed algorithm but the backend of that was changed and it started
    screwing up. I'm happier with random walker anyways though, I think it has better edges
    between QDs. 
    
    returns: (segmented image, #QDs)
    '''

    # threshholds the image and runs some openings and closings to make a mask
    threshholded_im = im > threshhold    
    opened_im = binary_opening(threshholded_im,iterations = 3)
    mask = binary_closing(opened_im, iterations = 3)
    
    # applies a distance transform and peak finder to get center of each QD
    distanceT = distance_transform_edt(mask)
    local_max_points = skimage.feature.corner_peaks(distanceT, min_distance=40, indices=True)
    other = skimage.feature.peak_local_max(distanceT, min_distance=40, indices=True)
    
    ''' So there is a small issue in the way skimage works: 
    peak_local_max labels all connected max points (I can't get it to not)
    and corner_peak labels the corner. So this adjusts that to the center 
    of the same-colored area within a cutoff range of 40 pixels. '''
    final_points = []
    cutoff_range = 40
    for point in local_max_points:
        py, px = point[0], point[1]
        color = distanceT[point[0], point[1]]
        others = np.where(distanceT == color)
        others_points = list(zip(others[0], others[1]))
        
        centers = np.zeros(np.shape(im))
        centers[int(py),int(px)] = 1
        for coord in others_points:
            cy, cx = coord[0], coord[1]
            dist = np.sqrt((cy-py)**2 + (cx - px)**2)
            if dist < cutoff_range:
                centers[int(cy), int(cx)] = 1
        
        # gets the center of the peak from distance transform via c.o.m. 
        com = center_of_mass(centers)
        final_points.append(com)
        

    # so at this point the user can see where the QD rough centers are, 
    # and add/remove as necessary.     
    check = 'n'
    while check != 'y':
        points_fixed = helpers.QD_check(im, final_points)
        check = str(input("does this work? (y/n)"))
    
    # now take the centers from points_fixed and turn it into a zeros array
    # with 1 at centers
    centers = np.zeros(np.shape(im))
    for point in final_points:
        centers[int(point[0]), int(point[1])] = 1
    
    # label to format with skimage and feed into random walker algorithm
    markers = skimage.measure.label(centers, return_num=True)
    marker_array = markers[0]
    marker_num = markers[1]
    
    marker_array[~mask] = -1
    rw_image = random_walker(mask, marker_array)

#     show_im(rw_image, "random walk")
    
    return(rw_image, marker_num)

In [30]:
from QD_state import QD
def center_masses( im, num, img_name):
    '''takes in the output of segment_im and gives you the centers of masses for all dat ish'''
    temp = []
    pixlist = []

    for i in range(1,int(num)+1):
        dot_im = np.where(im == i, 1, 0)
        npix = np.count_nonzero(dot_im)
        # pixlist.append(npix)

        cm = center_of_mass(dot_im)
        temp.append(cm)

    # get_histo(pixlist, 0, np.max(pixlist), 200)
    ret = []    
    ind = 0
    for i in temp:
        if not np.isnan(i[0]):
            ret.append(QD(i,img_name))
            ret[-1].QD_index = ind
        ind += 1 

    return(ret)

In [16]:
img_segmented = segment_im2(s97.orig_image, 16890)

adding [ 593.5086580086579 ,  18.339826839826742 ]
closing
These are the final ponts. 

            Click to close the figure and you'll be prompted 
            if you want to try again. 
does this work? (y/n)y


In [25]:
img_segmented[1]

27

In [31]:
QD_list = center_masses(img_segmented[0], img_segmented[1], "filename")

In [32]:
centers = []
for QD in QD_list:
    centers.append(QD.cm)

show_im_fits(s97.orig_image, np.array(centers))

In [24]:
np.shape(QD_list)

(24,)

In [26]:
QD_list[0].cm

(55.99249665455936, 321.90948193462054)

### spectral clustering

In [ ]:

from sklearn.feature_extraction import image
from sklearn.cluster import spectral_clustering

# l = 100
# x, y = np.indices((l, l))

# center1 = (28, 24)
# center2 = (40, 50)
# center3 = (67, 58)
# center4 = (24, 70)

# radius1, radius2, radius3, radius4 = 16, 14, 15, 14

# circle1 = (x - center1[0]) ** 2 + (y - center1[1]) ** 2 < radius1 ** 2
# circle2 = (x - center2[0]) ** 2 + (y - center2[1]) ** 2 < radius2 ** 2
# circle3 = (x - center3[0]) ** 2 + (y - center3[1]) ** 2 < radius3 ** 2
# circle4 = (x - center4[0]) ** 2 + (y - center4[1]) ** 2 < radius4 ** 2

# # #############################################################################
# # 4 circles
# img = circle1 + circle2 + circle3 + circle4

# # We use a mask that limits to the foreground: the problem that we are
# # interested in here is not separating the objects from the background,
# # # but separating them one from the other.

# img = img.astype(float)
# img += 1 + 0.2 * np.random.randn(*img.shape)


img = s97.orig_image
show_im(img, "img")
threshhold = 16890
threshholded_im = img > threshhold


opened_im = binary_opening(threshholded_im,iterations = 3)
mask = binary_closing(opened_im, iterations = 3)
# show_im(mask, "mask")

# Convert the image into a graph with the value of the gradient on the
# edges.
graph = image.img_to_graph(img, mask=mask)

print('graphed')
# Take a decreasing function of the gradient: we take it weakly
# dependent from the gradient the segmentation is close to a voronoi
graph.data = np.exp(-graph.data / graph.data.std())

print('graphed2')
# Force the solver to be arpack, since amg is numerically
# unstable on this example
labels = spectral_clustering(graph, n_clusters=4, eigen_solver='arpack')
print('labeled')
label_im = np.full(mask.shape, -1.)
label_im[mask] = labels

plt.matshow(img)
plt.matshow(label_im)

# #############################################################################
# # 2 circles
# img = circle1 + circle2
# mask = img.astype(bool)
# img = img.astype(float)

# img += 1 + 0.2 * np.random.randn(*img.shape)

# graph = image.img_to_graph(img, mask=mask)
# graph.data = np.exp(-graph.data / graph.data.std())

# labels = spectral_clustering(graph, n_clusters=2, eigen_solver='arpack')
# label_im = np.full(mask.shape, -1.)
# label_im[mask] = labels

# plt.matshow(img)
# plt.matshow(label_im)

# plt.show()

In [ ]:
100*100*8

In [ ]:
?graph

### Useful maybe for clicking things? 

In [ ]:
def test_cursor():
    import time



    def tellme(s):
        print(s)
        plt.title(s, fontsize=16)
        plt.draw()



    plt.clf()
    plt.axis([-1., 1., -1., 1.])
    plt.setp(plt.gca(), autoscale_on=False)

    tellme('You will define a triangle, click to begin')

    plt.waitforbuttonpress()

    while True:
        pts = []
        while len(pts) < 3:
            tellme('Select 3 corners with mouse')
            pts = np.asarray(plt.ginput(3, timeout=-1))
            if len(pts) < 3:
                tellme('Too few points, starting over')
                time.sleep(1)  # Wait a second

        ph = plt.fill(pts[:, 0], pts[:, 1], 'r', lw=2)

        tellme('Happy? Key click for yes, mouse click for no')

        if plt.waitforbuttonpress():
            break

        # Get rid of fill
        for p in ph:
            p.remove()

    # Define a nice function of distance from individual pts
    def f(x, y, pts):
        z = np.zeros_like(x)
        for p in pts:
            z = z + 1/(np.sqrt((x - p[0])**2 + (y - p[1])**2))
        return 1/z


    X, Y = np.meshgrid(np.linspace(-1, 1, 51), np.linspace(-1, 1, 51))
    Z = f(X, Y, pts)

    CS = plt.contour(X, Y, Z, 20)

    tellme('Use mouse to select contour label locations, middle button to finish')
    CL = plt.clabel(CS, manual=True)


    tellme('Now do a nested zoom, click to begin')
    plt.waitforbuttonpress()

    while True:
        tellme('Select two corners of zoom, middle mouse button to finish')
        pts = np.asarray(plt.ginput(2, timeout=-1))

        if len(pts) < 2:
            break

        pts = np.sort(pts, axis=0)
        plt.axis(pts.T.ravel())

    tellme('All Done!')
    plt.close()

In [ ]:
test_cursor()

In [ ]:
from matplotlib.widgets import Cursor


fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, facecolor='#FFFFCC')

x, y = 4*(np.random.rand(2, 100) - .5)
ax.plot(x, y, 'o')
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)

# set useblit = True on gtkagg for enhanced performance
cursor = Cursor(ax, useblit=True, color='red', linewidth=2)

plt.show()
